In [1]:
import easyocr

# Create an EasyOCR Reader instance
reader = easyocr.Reader(['en'])  # Specify the language(s)

# Perform OCR
def perform_ocr_easyocr(image_path):
    try:
        result = reader.readtext(image_path)
        extracted_text = " ".join([text[1] for text in result])  # Extract recognized text
        return extracted_text
    except Exception as e:
        print(f"Error during OCR: {e}")
        return None

# Example usage
image_path = "resources/test.png"  # Path to your image

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [4]:
text = perform_ocr_easyocr(image_path)
print("Extracted Text:", text)

Extracted Text: Q01. (25 Points) Locate the relative extrema for this function y =f(x) = 12 Answer: X-x?
